In [2]:
from sys import platform as _platform
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
#if _platform =='linux2':
#    path = '../data/data_sleep/' 
#else:
#    #mets ton path ici et ça devrait marcher :)
#    path = ""
path = "data/"

In [37]:
#frequencies=pd.read_csv(path+"data_frequences.csv").drop("Unnamed: 0", axis=1)
frequencies=pd.read_csv(path+"fft_eeg.csv")
frequencies_acc =pd.read_csv(path+"fft_acc.csv")

stats_acc=pd.read_csv(path+"stats_acc.csv")
stats_egg=pd.read_csv(path+"stats_eeg.csv")
stats=pd.concat([stats_acc, stats_egg], axis=1)
#ACC_feats =pd.read_csv(path+"ACC_feats.csv").drop("Unnamed: 0", axis=1)
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")
#stats = pd.concat([stats.filter(regex="EEG[0-9]*"),stats_ACC], axis=1)
#stats = pd.concat([stats,stats_ACC], axis=1)

frequencies_test=pd.read_csv(path+"fft_eeg_test.csv")
frequencies_acc_test =pd.read_csv(path+"fft_acc_test.csv")
ACC_feats_test =pd.read_csv(path+"ACC_feats_test.csv")
stats_acc_test=pd.read_csv(path+"stats_acc_test.csv")
stats_egg_test=pd.read_csv(path+"stats_eeg_test.csv")
stats_test=pd.concat([stats_acc_test, stats_egg_test], axis=1)

In [10]:
from sklearn.cross_validation import train_test_split
frequencies,  frequencies_test=train_test_split(frequencies, test_size=0.25, random_state=42)
frequencies_acc, frequencies_acc_test=train_test_split(frequencies_acc, test_size=0.25, random_state=42)

stats_acc, stats_acc_test=train_test_split(stats_acc, test_size=0.25, random_state=42)
stats_egg, stats_egg_test=train_test_split(stats_egg, test_size=0.25, random_state=42)
stats=pd.concat([stats_acc, stats_egg], axis=1)
del stats_acc, stats_egg
stats_test=pd.concat([stats_acc_test, stats_egg_test], axis=1)
del stats_acc_test, stats_egg_test
#ACC_feats =pd.read_csv(path+"ACC_feats.csv").drop("Unnamed: 0", axis=1)
labels, labels_test=train_test_split(labels, test_size=0.25, random_state=42)

In [39]:
def select_freq_names(low, high, X_columns, prefix = ''):
    return [name for name in X_columns 
            if len(name.split('q'))==2 
            and name.split('freq')[0] == prefix
            and low<=float(name.split('freq')[1]) 
            and high>= float(name.split('freq')[1])]
def group_frequencies(name, low, high, frequencies, prefix = ''):
    frequencies[name]=(1./(high-low) * (frequencies[select_freq_names(low,high,frequencies.columns,prefix)])).sum(axis=1)

In [40]:

#frequencies["delta"]=frequencies[select_freq_names(0,3,frequencies.columns)].sum(axis=1)
#frequencies["delta"]=frequencies[select_freq_names(0,3.99,frequencies.columns)].sum(axis=1)
#frequencies["theta"]=frequencies[select_freq_names(4,7.5,frequencies.columns)].sum(axis=1)
#frequencies["alpha"]=frequencies[select_freq_names(7.5,13.99,frequencies.columns)].sum(axis=1)
#frequencies["beta"]=frequencies[select_freq_names(14,50,frequencies.columns)].sum(axis=1)

def make_new_feats(frequencies):
    group_frequencies("delta1", 0.4, 0.8, frequencies)
    group_frequencies("delta2", 0.81, 1.99, frequencies)
    group_frequencies("delta3", 2, 3.99, frequencies)
    group_frequencies("theta1", 4, 5, frequencies)
    group_frequencies("theta2", 5, 7.499, frequencies)
    group_frequencies("alpha1", 7.5, 9.5, frequencies)
    group_frequencies("alpha2", 9.5, 10.499, frequencies)
    group_frequencies("alpha3", 10.5, 13.5, frequencies)
    group_frequencies("beta", 14, 20, frequencies)
    group_frequencies("beta2", 20, 30, frequencies)
    group_frequencies("beta3", 31, 50, frequencies)



In [41]:
def regroup_acc_freq (frequencies_acc):
    for prefix in ['ACC_X.','ACC_Y.','ACC_Z.']:
        group_frequencies(prefix+"smaller_one",0.01,1, frequencies_acc,prefix)
        group_frequencies(prefix+"one_to_two",1.01,2, frequencies_acc,prefix)
        group_frequencies(prefix+"two_to_three",2.01,3, frequencies_acc,prefix)
        group_frequencies(prefix+"three_to_four",3.01,4, frequencies_acc,prefix)
        group_frequencies(prefix+"more_four",4,10, frequencies_acc,prefix)




In [42]:
def make_new_features(f, stat, f_acc):
    new_feat = ["delta1","delta2","delta3", 'theta1', 'theta2', 'alpha1','alpha2','alpha3', 'beta','beta2','beta3']
    make_new_feats(f)
    
    prefixes = ['ACC_X.','ACC_Y.','ACC_Z.']
    regroup_acc_freq (f_acc)
    frequencies_acc2 = f_acc[[prefix+ x for x in["smaller_one","one_to_two","two_to_three",'more_four']for prefix in prefixes]]
    
    return  pd.concat([f[new_feat], stat, frequencies_acc2], axis=1)

In [43]:
train = make_new_features(frequencies, stats, frequencies_acc)
test = make_new_features(frequencies_test, stats_test, frequencies_acc_test)

In [44]:
train.describe()

,delta1,delta2,delta3,theta1,theta2,alpha1,alpha2,alpha3,beta,beta2,beta3,ACC_X_quantile_0,ACC_X_quantile_10,ACC_X_quantile_20,ACC_X_quantile_30,ACC_X_quantile_40,ACC_X_quantile_50,ACC_X_quantile_60,ACC_X_quantile_70,ACC_X_quantile_80,ACC_X_quantile_90,ACC_X_quantile_100,mean_ACC_X,var_ACC_X,skew_ACC_X,kurt_ACC_X,ACC_Y_quantile_0,ACC_Y_quantile_10,ACC_Y_quantile_20,ACC_Y_quantile_30,ACC_Y_quantile_40,ACC_Y_quantile_50,ACC_Y_quantile_60,ACC_Y_quantile_70,ACC_Y_quantile_80,ACC_Y_quantile_90,ACC_Y_quantile_100,mean_ACC_Y,var_ACC_Y,skew_ACC_Y,kurt_ACC_Y,ACC_Z_quantile_0,ACC_Z_quantile_10,ACC_Z_quantile_20,ACC_Z_quantile_30,ACC_Z_quantile_40,ACC_Z_quantile_50,ACC_Z_quantile_60,ACC_Z_quantile_70,ACC_Z_quantile_80,ACC_Z_quantile_90,ACC_Z_quantile_100,mean_ACC_Z,var_ACC_Z,skew_ACC_Z,kurt_ACC_Z,EEG_quantile_0,EEG_quantile_10,EEG_quantile_20,EEG_quantile_30,EEG_quantile_40,EEG_quantile_50,EEG_quantile_60,EEG_quantile_70,EEG_quantile_80,EEG_quantile_90,EEG_quantile_100,mean_EEG,var_EEG,skew_EEG,kurt_EEG,EEG_through_0,ACC_X.smaller_one,ACC_Y.smaller_one,ACC_Z.smaller_one,ACC_X.one_to_two,ACC_Y.one_to_two,ACC_Z.one_to_two,ACC_X.two_to_three,ACC_Y.two_to_three,ACC_Z.two_to_three,ACC_X.more_four,ACC_Y.more_four,ACC_Z.more_four
count,3.112900e+04,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,3.112900e+04,3.105900e+04,3.112900e+04,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,3.112900e+04,31095.000000,3.112900e+04,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,3.112900e+04,3.112700e+04,3.112900e+04,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,3.112900e+04,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000,31129.000000
mean,2.180787e+05,66415.984901,25777.882487,16727.005791,12412.698942,10436.192508,9230.629203,7652.617749,4395.367835,3076.966534,2217.256648,463043.459858,467174.530536,474999.947300,492067.066492,512800.096769,534592.475148,556714.590114,578462.955919,599379.979357,614626.710998,619450.027578,537272.507188,3.330232e+10,-9.025399e+04,-4.718321e+13,508458.001608,512433.331547,520669.693936,538912.377147,560303.097796,583160.354622,606010.389134,628979.842747,650082.743408,665331.568056,670117.303174,585548.917565,3.488588e+10,-44216.596762,7.224458e+13,389079.403871,394942.680469,401722.919196,418578.215755,441054.816154,464406.560797,487958.291960,511829.372506,533244.536975,548457.120201,553119.307859,467403.641997,3.551936e+10,-1.056223e+05,4.234130e+12,-67.668904,-24.578816,-15.029817,-9.142491,-4.553969,-0.429837,3.709411,8.344859,14.295154,23.852064,70.926835,-0.307724,8.091409e+03,0.042390,1.888080,266.303897,55257.764563,59060.369738,68198.799131,45070.254993,46029.634280,52876.211177,58954.750214,60437.381404,57634.399315,6722.761692,6419.831198,7002.951855
std,1.049126e+06,401127.379008,193613.862060,130811.552912,90632.399730,65039.692153,55269.180990,46207.481534,33265.198819,22897.909771,14408.527366,586520.188548,587923.715579,583843.126280,575799.732902,571903.857252,576135.801496,588850.829758,609066.069517,634782.817990,656140.274833,661408.888219,576190.718726,1.071621e+11,3.791622e+06,1.858705e+15,670905.568374,672213.290684,668321.467589,660155.944471,655888.106042,658494.868185,668761.620419,686526.242650,708558.016987,726915.544490,729208.429707,657677.633921,1.104874e+11,2012175.965070,2.258636e+15,561598.622147,565504

In [45]:
test.describe()

,delta1,delta2,delta3,theta1,theta2,alpha1,alpha2,alpha3,beta,beta2,beta3,ACC_X_quantile_0,ACC_X_quantile_10,ACC_X_quantile_20,ACC_X_quantile_30,ACC_X_quantile_40,ACC_X_quantile_50,ACC_X_quantile_60,ACC_X_quantile_70,ACC_X_quantile_80,ACC_X_quantile_90,ACC_X_quantile_100,mean_ACC_X,var_ACC_X,skew_ACC_X,kurt_ACC_X,ACC_Y_quantile_0,ACC_Y_quantile_10,ACC_Y_quantile_20,ACC_Y_quantile_30,ACC_Y_quantile_40,ACC_Y_quantile_50,ACC_Y_quantile_60,ACC_Y_quantile_70,ACC_Y_quantile_80,ACC_Y_quantile_90,ACC_Y_quantile_100,mean_ACC_Y,var_ACC_Y,skew_ACC_Y,kurt_ACC_Y,ACC_Z_quantile_0,ACC_Z_quantile_10,ACC_Z_quantile_20,ACC_Z_quantile_30,ACC_Z_quantile_40,ACC_Z_quantile_50,ACC_Z_quantile_60,ACC_Z_quantile_70,ACC_Z_quantile_80,ACC_Z_quantile_90,ACC_Z_quantile_100,mean_ACC_Z,var_ACC_Z,skew_ACC_Z,kurt_ACC_Z,EEG_quantile_0,EEG_quantile_10,EEG_quantile_20,EEG_quantile_30,EEG_quantile_40,EEG_quantile_50,EEG_quantile_60,EEG_quantile_70,EEG_quantile_80,EEG_quantile_90,EEG_quantile_100,mean_EEG,var_EEG,skew_EEG,kurt_EEG,EEG_through_0,ACC_X.smaller_one,ACC_Y.smaller_one,ACC_Z.smaller_one,ACC_X.one_to_two,ACC_Y.one_to_two,ACC_Z.one_to_two,ACC_X.two_to_three,ACC_Y.two_to_three,ACC_Z.two_to_three,ACC_X.more_four,ACC_Y.more_four,ACC_Z.more_four
count,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,3.045800e+04,30458.000000,3.045800e+04,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,3.045800e+04,3.045800e+04,3.045800e+04,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,3.045800e+04,3.045700e+04,3.045800e+04,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000,30458.000000
mean,51.873624,17.992502,6.794295,4.340244,3.167898,2.499432,2.196317,1.872309,1.083713,0.781358,0.563860,563610.290791,567084.633856,577435.056470,601765.952872,635351.414288,671050.579973,707440.210060,744038.631187,779255.487619,807892.068274,815905.409883,678133.310874,5.457041e+10,-11410.973991,6.644445e+11,552288.799338,557927.042470,572286.779482,601085.387311,635182.176917,670568.047277,706392.329323,742101.057054,775371.803300,802288.424375,810697.845677,674523.009937,5.458737e+10,-6.816553e+03,6.818055e+11,465696.911817,469407.635961,479328.821356,503049.706907,537145.351262,573888.496563,611873.577913,649789.417138,685949.469918,713448.177243,723819.286615,581850.570548,5.574086e+10,-5.445095e+04,7.059670e+12,-63.373048,-22.649193,-14.037461,-8.625256,-4.343720,-0.455796,3.454877,7.798586,13.280475,22.032783,64.157176,-0.329917,618.463526,0.059365,1.787492,295.440935,88299.482826,88500.440793,95394.274575,81514.311799,73358.578653,89083.058384,92921.692146,89994.768631,98861.250739,10914.984852,10595.137854,11014.606995
std,54.592059,13.205135,3.736462,2.131689,1.404761,1.241717,1.140811,0.832338,0.544895,0.509729,0.592141,619457.702821,619562.521660,612280.442131,596534.214003,581737.122153,577926.703959,587184.991182,609209.818331,640521.195853,671721.534660,678037.978785,576678.553613,1.328382e+11,808430.175061,4.807920e+13,681836.912157,683381.254168,675428.557810,659786.874618,649089.426501,648333.716867,658702.353746,679649.824010,707579.091081,734768.551478,740065.079932,646824.532816,1.317086e+11,8.410267e+05,8.413946e+13,611918.188137,613091.055042,608077.877470,596350.094318,586209.146948,587664.316146,602988.678536,63101

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
y_train = labels['TARGET']
y_test = labels_test['TARGET']
X = train.drop(["skew_ACC_X","skew_ACC_Y", "skew_ACC_Z"], axis=1)
X_t = test.drop(["skew_ACC_X","skew_ACC_Y", "skew_ACC_Z"], axis=1)
#X, X_t, y_train, y_test = train_test_split(
#    X, labels["TARGET"], test_size=0.2)

scaler = StandardScaler().partial_fit(X)
scaler = StandardScaler().partial_fit(X_t)
X_train = scaler.transform(X)
X_test = scaler.transform(X_t)
X_columns = train.columns

In [ ]:
%%time
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import cohen_kappa_score
et = ExtraTreesClassifier(random_state = 42,n_estimators=1000, max_depth= 29, n_jobs = 3, class_weight = 'balanced')
et.fit(X_train, y_train)
predicted_label = et.predict(X_test)

#print("Extra Random Trees - accuracy Score on test_data : ", accuracy_score(y_test, predicted_label))
print("Extra Random Trees - kappa Score on test_data : ", cohen_kappa_score(y_test, predicted_label))
print("Extra Random Trees - kappa Score on train data : ", cohen_kappa_score(y_train, et.predict(X_train)))

In [ ]:
#from sklearn.metrics import accuracy_score
#print("Extra Random Trees - accuracy Score on test_data : ", accuracy_score(y_test, predicted_label))
#print("Extra Random Trees - kappa Score on test_data : ", cohen_kappa_score(y_test, predicted_label))

In [ ]:
plt.figure(figsize=(15, 5))

ordering = np.argsort(et.feature_importances_)[::-1][:50]

importances = et.feature_importances_[ordering]
feature_names = X_columns[ordering]

x = np.arange(len(feature_names))
plt.bar(x, importances)
plt.xticks(x + 0.5, feature_names, rotation=90, fontsize=15);

In [30]:
%%time
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(et, X_train, y_train, cv=5, scoring="accuracy",n_jobs=3)
print("accuracy: {:.4f} +/-{:.4f}".format(
    np.mean(scores), np.std(scores)))

KeyboardInterrupt: 

In [32]:
labels_test.head()

,ID,TARGET
3826,ID03827,2
23673,ID23674,2
18304,ID18305,2
2943,ID02944,3
8158,ID08159,3


In [ ]:
def make_submission(predicted_label, name = 'submit.csv'):
    ids=labels_test["ID"]
    ids = pd.read_csv('../data/data_sleep/input_test.csv', usecols= ['ID'])
    ids["TARGET"] = pd.Series(predicted_label)
    ids['TARGET']= ids['TARGET'].astype(int)
    ids.to_csv(name,index=False)
    #return ids

In [ ]:
make_submission(predicted_label)

In [35]:
ids.head()

3826     ID03827
23673    ID23674
18304    ID18305
2943     ID02944
8158     ID08159
Name: ID, dtype: object

In [ ]:
print("Extra Random Trees - kappa Score on test_data : ", cohen_kappa_score(y_test, predicted_label))

In [16]:
predicted_label_train = et.predict(X_train)

In [17]:
import collections
print collections.Counter(predicted_label), collections.Counter(y_train)

Counter({2.0: 26010, 4.0: 2276, 3.0: 1598, 0.0: 493, 1.0: 81}) Counter({2.0: 15334, 3.0: 9640, 4.0: 4385, 0.0: 1342, 1.0: 428})


In [18]:
X_train.shape, X_test.shape

((31129, 81), (30458, 81))